In [ ]:
DevEnvironment = False
if not DevEnvironment:
  !pip install swcc tqdm
else:
  # If you have the source code locally (say on /src/shapeworks-cloud), you can install from there:
  !pip install tqdm
  !pip install -e /src/shapeworks-cloud/swcc/.

In [2]:
# We can increase logging
import logging
logging.getLogger('swcc').setLevel(logging.INFO)

In [3]:
from pathlib import Path
import re
from shutil import rmtree

import getpass
from tqdm.notebook import tqdm

from swcc.api import swcc_session
from swcc.models import (
    Dataset, GroomedSegmentation, OptimizedParticles,
    OptimizedShapeModel, Project, Segmentation, Subject
)

In [4]:
if not DevEnvironment:
    with swcc_session() as session:
        token = session.login(getpass.getpass('username'), getpass.getpass('password'))

        print([(d.id, d.name) for d in Dataset.list()])

In [5]:
if not DevEnvironment:
    # For development, it is possible to generate a session outside of a context manager.
    ctx = swcc_session(token=token)
    session = ctx.__enter__()

    print([(d.id, d.name) for d in Dataset.list()]) 

In [6]:
# To connect to a different server, sepecify it as part of the session:
if DevEnvironment:
    base_url = 'http://burgess:8000/api/v1'
    with swcc_session(base_url=base_url) as session:
        token = session.login('admin@noemail.com', 'django-password')
    ctx = swcc_session(base_url=base_url, token=token)
    session = ctx.__enter__()


In [7]:
# All domain models are pydantic models that can be created in offline mode:
name = 'test dataset'
old = Dataset.from_name(name)
if old:
    old.delete()
# To generate the entity on the server, you must call its `create` method
dataset = Dataset(name=name, license='license', description='just a test', acknowledgement='NIH').create()

dataset

Dataset(id=132, name='test dataset', file=None, license='license', description='just a test', acknowledgement='NIH', keywords='', contributors='', publications='')

In [8]:
# Given an id, you can fetch data from the server
dataset_id = dataset.id
Dataset.from_id(dataset_id)

Dataset(id=132, name='test dataset', file=None, license='license', description='just a test', acknowledgement='NIH', keywords='', contributors='', publications='')

In [9]:
# You can also list all entities on the server using the `list` method
for d in Dataset.list():
    print(d)

id=131 name='left_atrium' file=<swcc.models.FileType object at 0x7f6e0d961af0> license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross Whitaker. "Shapeworks: particle-based shape correspondence and visualization software." Statistical Shape and Deformation Analy

In [10]:
# Entities are deleted from the server using the `delete` method
# dataset.delete()

# Now we find it no longer exists
Dataset.from_id(dataset_id)

Dataset(id=132, name='test dataset', file=None, license='license', description='just a test', acknowledgement='NIH', keywords='', contributors='', publications='')

A dataset consists of an Excel .xlsx file with a sheet named `data`.  That sheet has to contain two columns: `shape_file` and `image_file`, both of which contain corresponding lists of relative paths to .nrrd files.

Any number of projects can be aded to a dataset.  Each project consists of an Excel .xlsx file with two sheets: `data` and `optimize`.  The `data` sheet has columns `shape_file`, `groomed_file`, `alignment_file`, `local_particles_file`, and `world_particles_file`.  The shape file and groomed file columns contain relative paths to .nrrd files.  The local and world particles file columns contain relative paths to .particles files.  The alignment file column is a 12 parameter transform consisting of numbers separated by spaces.  The `optimize` sheet has columnes `key` and `value` which contain model parameters.

In [11]:
# upload the left_atrium dataset
id_re = re.compile(r'.*(CARMA\d+)\.[^/]*')
root_path = Path('data')
dataset_path = root_path / 'left_atrium-v0'

basename = 'left_atrium'
name = basename
# This is no longer necessary if you use force_create
if False:
    suffix = 0
    while True:
        old = Dataset.from_name(name)
        # Either get rid of an existing dataset with the same name, or create a new name for testing
        if not old:
            break
        if False:
            print('Deleting previous version of %s' % name)
            old.delete()
        suffix = suffix + 1
        name = '%s-%s' % (basename, suffix)
# TODO: Fill in with correct information
description = 'left_atrium'
acknowledgement = 'acknowledgement'

with (dataset_path / 'License.txt').open('r') as f:
    license = f.read()

dataset = Dataset(name=name, file=dataset_path / 'data.xlsx', license=license, description=description, acknowledgement=acknowledgement).force_create(backup=False)
dataset.add_project(dataset_path / Path('project.xlsx'))
print(dataset.id, dataset.name)

[20:40:50] INFO     Uploading file data.xlsx                                    ]8;id=254009;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=292235;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

[20:40:51] INFO     Uploading file CARMA0046.laendo_no_veins.nrrd               ]8;id=521328;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=819601;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

           INFO     Uploading file CARMA0046.lgemri.nrrd                        ]8;id=845026;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=371166;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

[20:40:52] INFO     Uploading file CARMA0072.laendo_no_veins.nrrd               ]8;id=329363;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=860479;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

           INFO     Uploading file CARMA0072.lgemri.nrrd                        ]8;id=307851;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=445149;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

...

           INFO     Uploading file CARMA0046.laendo_no_veins.isores.center.pad. ]8;id=285225;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=261669;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\
                    com.aligned.cropped.tpSmoothDT.nrrd

[20:42:31] INFO     Uploading file CARMA0046.laendo_no_veins.isores.center.pad. ]8;id=895773;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=222631;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\
                    com.aligned.cropped.tpSmoothDT_world.particles

           INFO     Uploading file CARMA0046.laendo_no_veins.isores.center.pad. ]8;id=617691;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=818931;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\
                    com.aligned.cropped.tpSmoothDT_local.particles

           INFO     Uploading file CARMA0046.laendo_no_veins.transform          ]8;id=34631;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=899015;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

133 left_atrium


In [12]:
# We can fetch a dataset by name
d = Dataset.from_name(name)
print(d.dict())

{'id': 133, 'name': 'left_atrium', 'file': <swcc.models.FileType object at 0x7f6e0c094d60>, 'license': '=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross Whitaker. "Shapeworks: particle-based shape correspondence and visualization software." Statistical Shape and D

In [13]:
# Print information about the dataset
for key in ('acknowledgement', 'contributors', 'description', 'id', 'keywords', 'license', 'name', 'publications'):
    print('%s: %s' % (key, getattr(dataset, key)))

acknowledgement: acknowledgement
contributors: 
description: left_atrium
id: 133
keywords: 
license: === NAMIC: CARMA Left Atrial MRI ===

If you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the 'Acknowledgments' section of your paper:

"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."

and add the following 'disclaimer':

"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."


When referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.

Joshua Cates, Shireen Elhabian, Ross Whitaker. "Shapeworks: particle-based shape correspondence and visualization software." Statistical Shape and Deformation Analysis.

In [14]:
# Some components of the dataset are returned from functions
for key in ('dict', 'schema', 'schema_json'):  # 'json'
    print('%s: %s' % (key, getattr(dataset, key)()))
# Some components are generators    
for key in ('projects', 'segmentations', 'subjects', ): 
    for idx, value in enumerate(getattr(dataset, key)):
        print('%s:%d: %s' % (key, idx, value))


dict: {'id': 133, 'name': 'left_atrium', 'file': <swcc.models.FileType object at 0x7f6e0c0407f0>, 'license': '=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross Whitaker. "Shapeworks: particle-based shape correspondence and visualization software." Statistical Shape

segmentations:0: id=2529 file=<swcc.models.FileType object at 0x7f6e0d91a130> anatomy_type='unknown' subject=Subject(id=2533, name='CARMA0046.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sc

segmentations:3: id=2532 file=<swcc.models.FileType object at 0x7f6e0d91a190> anatomy_type='unknown' subject=Subject(id=2536, name='CARMA0254.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sc

segmentations:6: id=2535 file=<swcc.models.FileType object at 0x7f6e0d91a520> anatomy_type='unknown' subject=Subject(id=2539, name='CARMA0301.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sc

segmentations:9: id=2538 file=<swcc.models.FileType object at 0x7f6e0d91a100> anatomy_type='unknown' subject=Subject(id=2542, name='CARMA0415.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sc

...

segmentations:37: id=2566 file=<swcc.models.FileType object at 0x7f6e0d94b280> anatomy_type='unknown' subject=Subject(id=2570, name='CARMA1296.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.s

segmentations:40: id=2569 file=<swcc.models.FileType object at 0x7f6e07fd5250> anatomy_type='unknown' subject=Subject(id=2573, name='CARMA1311.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.s

segmentations:43: id=2572 file=<swcc.models.FileType object at 0x7f6e07fcfac0> anatomy_type='unknown' subject=Subject(id=2576, name='CARMA1331.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.s

segmentations:46: id=2575 file=<swcc.models.FileType object at 0x7f6e0d94b5b0> anatomy_type='unknown' subject=Subject(id=2579, name='CARMA1398.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.s

segmentations:49: id=2578 file=<swcc.models.FileType object at 0x7f6e0d94b0d0> anatomy_type='unknown' subject=Subject(id=2582, name='CARMA1404.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.s

In [15]:
# We can examine a segmentation
seg = next(dataset.segmentations)
print('id: %s\nsubject: %s\nfile: %s\nanatomy_type: %s' % (seg.id, seg.subject.name, seg.file.name, seg.anatomy_type))
# We could also get the information as a dictionary or json via:
# seg.dict() or seg.json()

id: 2529
subject: CARMA0046.laendo_no_veins
file: CARMA0046.laendo_no_veins.nrrd
anatomy_type: unknown


In [16]:
# We can examine a subject
subj = next(dataset.subjects)
# We could have also gotten the subject of a segmentation:
# subj = next(dataset.segmentation).subject
print('id: %s\nname: %s\nsegmentations: %s' % (subj.id, subj.name, list(subj.segmentations)))


id: 2533
name: CARMA0046.laendo_no_veins
segmentations: [Segmentation(id=2529, file=<swcc.models.FileType object at 0x7f6e0d91a880>, anatomy_type='unknown', subject=Subject(id=2533, name='CARMA0046.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper 

In [17]:
# We can examine a project
proj = next(dataset.projects)
print('id: %s\nfile: %s\ndescription: %s\ngroomed segmentations: %s\nshape models: %s' % (
    proj.id, proj.file.name, description, list(proj.groomed_segmentations), list(proj.shape_models)))
# And a groomed segmentaion
print('------ groomed segmentation ------')
gseg = next(proj.groomed_segmentations)
print(gseg.dict())
# Or a shape models
print('------ shape model ------')
smod = next(proj.shape_models)
print(smod.dict())

id: 40
file: project.xlsx
description: left_atrium
groomed segmentations: [GroomedSegmentation(id=2578, file=<swcc.models.FileType object at 0x7f6e07fcfe80>, pre_cropping=None, pre_alignment=None, segmentation=Segmentation(id=2578, file=<swcc.models.FileType object at 0x7f6e0d91a190>, anatomy_type='unknown', subject=Subject(id=2582, name='CARMA1404.laendo_no_veins', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c0407f0>, license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily repre

In [18]:
xls = dataset.download('/tmp/example')

[20:42:43] INFO     Downloading /tmp/example/data.xlsx                          ]8;id=92037;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=892452;file:///src/shapeworks-cloud/swcc/swcc/models.py#173\173]8;;\

           INFO     Downloading /tmp/example/project.xlsx                       ]8;id=912121;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=915835;file:///src/shapeworks-cloud/swcc/swcc/models.py#173\173]8;;\

[20:43:48] INFO     Downloading /tmp/example/groomed/distance_transforms/CARMA1 ]8;id=811325;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=972990;file:///src/shapeworks-cloud/swcc/swcc/models.py#173\173]8;;\
                    404.laendo_no_veins.isores.center.pad.com.aligned.cropped.t              
                    pSmoothDT.nrrd

           INFO     Downloading /tmp/example/shape_models/single_scale/512/CARM ]8;id=275518;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=724360;file:///src/shapeworks-cloud/swcc/swcc/models.py#173\173]8;;\
                    A1404.laendo_no_veins.isores.center.pad.com.aligned.cropped              
                    .tpSmoothDT_local.particles

           INFO     Downloading /tmp/example/shape_models/single_scale/512/CARM ]8;id=351557;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=624707;file:///src/shapeworks-cloud/swcc/swcc/models.py#173\173]8;;\
                    A1404.laendo_no_veins.isores.center.pad.com.aligned.cropped              
                    .tpSmoothDT_world.particles

...

           INFO     Downloading /tmp/example/images/CARMA1399.lgemri.nrrd       ]8;id=110059;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=294905;file:///src/shapeworks-cloud/swcc/swcc/models.py#173\173]8;;\

           INFO     Downloading                                                 ]8;id=600496;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=671362;file:///src/shapeworks-cloud/swcc/swcc/models.py#173\173]8;;\
                    /tmp/example/segmentations/CARMA1401.laendo_no_veins.nrrd

[20:44:12] INFO     Downloading /tmp/example/images/CARMA1401.lgemri.nrrd       ]8;id=544664;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=742207;file:///src/shapeworks-cloud/swcc/swcc/models.py#173\173]8;;\

           INFO     Downloading                                                 ]8;id=503977;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=973862;file:///src/shapeworks-cloud/swcc/swcc/models.py#173\173]8;;\
                    /tmp/example/segmentations/CARMA1404.laendo_no_veins.nrrd

           INFO     Downloading /tmp/example/images/CARMA1404.lgemri.nrrd       ]8;id=263802;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=490247;file:///src/shapeworks-cloud/swcc/swcc/models.py#173\173]8;;\

In [19]:
dataset_path = '/tmp/example'
root_path = Path(dataset_path)

# TODO: Fill in with correct information
name = description = 'left_atrium_test_2'
old = Dataset.from_name(name)
if old:
  print('Deleting previous version of %s' % name)
  old.delete()

acknowledgement = 'acknowledgement'
license = 'license'

d2 = Dataset(name=name, file=root_path / 'data.xlsx', license=license, description=description, acknowledgement=acknowledgement).create()
d2.add_project(root_path / 'project.xlsx')

Deleting previous version of left_atrium_test_2


[20:44:13] INFO     Uploading file data.xlsx                                    ]8;id=328631;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=247146;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

           INFO     Uploading file CARMA0046.laendo_no_veins.nrrd               ]8;id=606091;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=755158;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

           INFO     Uploading file CARMA0046.lgemri.nrrd                        ]8;id=621021;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=806708;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

[20:44:14] INFO     Uploading file CARMA0072.laendo_no_veins.nrrd               ]8;id=32063;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=789613;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

...

           INFO     Uploading file CARMA0046.laendo_no_veins.isores.center.pad. ]8;id=449084;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=590726;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\
                    com.aligned.cropped.tpSmoothDT.nrrd

[20:45:54] INFO     Uploading file CARMA0046.laendo_no_veins.isores.center.pad. ]8;id=702262;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=509254;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\
                    com.aligned.cropped.tpSmoothDT_world.particles

           INFO     Uploading file CARMA0046.laendo_no_veins.isores.center.pad. ]8;id=802426;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=716365;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\
                    com.aligned.cropped.tpSmoothDT_local.particles

           INFO     Uploading file CARMA0046.laendo_no_veins.transform          ]8;id=810557;file:///src/shapeworks-cloud/swcc/swcc/models.py\models.py]8;;\:]8;id=876450;file:///src/shapeworks-cloud/swcc/swcc/models.py#149\149]8;;\

Project(id=41, file=<swcc.models.FileType object at 0x7f6e07c2ad30>, keywords='', description='', dataset=Dataset(id=134, name='left_atrium_test_2', file=<swcc.models.FileType object at 0x7f6e07c2a400>, license='license', description='left_atrium_test_2', acknowledgement='acknowledgement', keywords='', contributors='', publications=''))

When accessing a dataset from the portal, the projects can be accessed:

In [20]:
dataset = Dataset.from_name('left_atrium')
# dataset.projects is a generator that will step through all projects in the dataset
print([proj.file.name for proj in dataset.projects])
# shape models are accessed through a generator on the project
# get a reference to the first project
proj = next(dataset.projects)
print([model.parameters for model in proj.shape_models])
print(next(next(proj.shape_models).particles))


['project.xlsx']
[{'multiscale': 'false', 'procrustes': 'true', 'use_normals': 1.0, 'normals_strength': 10.0, 'procrustes_scaling': 'true', 'relative_weighting': 10.0, 'number_of_particles': 512.0, 'procrustes_interval': 1.0, 'iterations_per_split': 4000.0, 'multiscale_particles': 32.0, 'ending_regularization': 10.0, 'use_geodesic_distance': 'false', 'optimize_output_prefix': '<project>_particles', 'optimization_iterations': 4000.0, 'starting_regularization': 1000.0, 'geodesic_cache_multiplier': 0.0, 'initial_relative_weighting': 0.1}]
id=1760 world=<swcc.models.FileType object at 0x7f6e0c048e20> local=<swcc.models.FileType object at 0x7f6e07ccd7f0> transform=<swcc.models.FileType object at 0x7f6e07bda340> shape_model=OptimizedShapeModel(id=37, project=Project(id=40, file=<swcc.models.FileType object at 0x7f6e0d961b20>, keywords='', description='', dataset=Dataset(id=133, name='left_atrium', file=<swcc.models.FileType object at 0x7f6e0c048f40>, license='=== NAMIC: CARMA Left Atrial MRI